Imports

In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import loader
import tables_to_leave
from sklearn.externals.joblib import dump, load
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

Paths

In [7]:
KNOWN_PATCH = 'C:\\Users\\Tom\\PycharmProjects\\wiseNeuro\\data\\known.csv'
SKY_PATCH = 'C:\\Users\\Tom\\PycharmProjects\\wiseNeuro\\data\\sky_train.csv'
TEST_PATH = ''

Loading data files

In [13]:
known_df = loader.load_file(KNOWN_PATCH, sep=';')
known_df['isdwarf'] = 1
known_df = loader.drop_tables(known_df, tables_to_leave.TABLE_LIST)
sky_df = loader.load_file(SKY_PATCH, sep=',')
sky_df['isdwarf'] = 0
sky_df = loader.drop_tables(sky_df, tables_to_leave.TABLE_LIST)
df = pd.concat([known_df, sky_df], sort=False)
df = df.fillna(0)
df_len = len(df.columns)

Creating dataset

In [14]:
target = df.pop('isdwarf')
scaler = StandardScaler()
scaler.fit(df)
dump(scaler, 'std_scaler.bin', compress=True)
df = scaler.transform(df)
dataset = tf.data.Dataset.from_tensor_slices((df, target.values))
train_dataset = dataset.shuffle(len(df)).batch(100)

Creating model

In [18]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(df_len, activation='sigmoid'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(200, activation='tanh'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(400, activation='sigmoid'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    optim = tf.keras.optimizers.SGD(learning_rate=0.01)
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer=optim,
                  loss=loss_fn,
                  metrics=['accuracy'])
    return model

Running model

In [19]:
model = get_compiled_model()
model.fit(train_dataset, epochs=10)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

920/920 [==============================] - 2s 2ms/step - loss: 0.1022 - accuracy: 0.9792
Epoch 2/10
920/920 [==============================] - 2s 2ms/step - loss: 0.0993 - accuracy: 0.9795
Epoch 3/10
920/920 [==============================] - 2s 2ms/step - loss: 0.0989 - accuracy: 0.9795
Epoch 4/10
920/920 [==============================] - 2s 2ms/step - loss: 0.0986 - accuracy: 0.9795
Epoch 5/10
920/920 [==============================] - 2s 2ms/step - loss: 0.0983 - accuracy: 0.9795
Epoch 6/

Saving data

In [20]:
loader.save_model(model)

Saved model to disk
